### Process CLUSTER files (e.g. WHISPER) and outputs as netCDF

In [1]:
import pandas as pd
import numpy as np
import cdflib
from sgp4.api import Satrec, jday
from datetime import datetime
import xarray as xr
import re
import os

Need to change data['example-z-var']

In [2]:
dir_path = "./electron-density"
files = os.listdir(dir_path)

file_in = cdflib.CDF(os.path.join(dir_path, files[0]))

file_info = file_in.cdf_info()
zvarinfo = file_info.zVariables

znum = []
zvar = []
zdim = []
zfill = []
zdtype = []
zunits = []

for i in zvarinfo:
    ivar = file_in.varinq(i)
    znum.append(ivar.Num)
    zvar.append(ivar.Variable)
    zdim.append(ivar.Dim_Sizes)
    zdtype.append(ivar.Data_Type_Description)
    try:
        zfill.append(file_in.varattsget(i)['FILLVAL'])
        zunits.append(file_in.varattsget(i)['UNITS'])
    except KeyError:
        zfill.append('NaN')
        zunits.append('NaN')
        continue

pd.DataFrame({'zvar': zvar, 'zdim': zdim, 'ztype': zdtype, 'zfill': zfill, 'zunits': zunits})

,zvar,zdim,ztype,zfill,zunits
0,time_tags__C2_CP_WHI_ELECTRON_DENSITY,[],CDF_EPOCH,315569519999000.0,ms
1,Delta_Time__C2_CP_WHI_ELECTRON_DENSITY,[],CDF_FLOAT,-1.0,s
2,Spectrum_Type__C2_CP_WHI_ELECTRON_DENSITY,[],CDF_CHAR,-,unitless
3,Computation_Method__C2_CP_WHI_ELECTRON_DENSITY,[],CDF_CHAR,XX,unitless
4,External_Data__C2_CP_WHI_ELECTRON_DENSITY,[],CDF_CHAR,-,unitless
5,Human_Validation__C2_CP_WHI_ELECTRON_DENSITY,[],CDF_CHAR,-,unitless
6,Electron_Density__C2_CP_WHI_ELECTRON_DENSITY,[],CDF_FLOAT,-1.0,cm^-3
7,Uncertainty__C2_CP_WHI_ELECTRON_DENSITY,[],CDF_FLOAT,-1.0,cm^-3
8,Contrast__C2_CP_WHI_ELECTRON_DENSITY,[],CDF_FLOAT,-1.0,unitless
9,Quality__C2_CP_WHI_ELECTRON_DENSITY,[],CDF_INT4,-1,unitless


In [3]:
%%time

ed = np.zeros((1,), dtype='object')
dt = np.zeros((1,), dtype='object')

for file in files:
    if file.endswith(".cdf"):
        file_path = os.path.join(dir_path, file)
        data_store = cdflib.CDF(file_path)
        time_store = data_store[zvar[0]]
        dt_store = pd.to_datetime(cdflib.cdfepoch.to_datetime(time_store))
        dt = np.append(dt, dt_store, axis=0)
        ed_store = data_store[zvar[6]]
        ed = np.append(ed, ed_store, axis=0)

dt = dt[1:]
dt[dt == float(zfill[0])] = 'NaN'

ed = ed[1:]
ed[ed == float(zfill[6])] = 'NaN'

out = xr.DataArray(data = ed, dims = ("t"), coords = {"t": dt}, name = 'n_e')
out.to_netcdf('whi_n_e.nc')

CPU times: total: 25.2 s
Wall time: 25.5 s


In [54]:
dir_path = "./natural"
files = os.listdir(dir_path)
i=0
for file in files:
    i+=1
    if file.endswith(".cdf"):
        file_path = os.path.join(dir_path, file)
        data = cdflib.CDF(file_path)
        espd = data['Electric_Spectral_Power_Density__C2_CP_WHI_NATURAL']
        time = data['time_tags__C2_CP_WHI_NATURAL']
        freq = data['Spectral_Frequencies__C2_CP_WHI_NATURAL']
        dt = pd.to_datetime(cdflib.cdfepoch.to_datetime(time))
        out = xr.DataArray(espd, dims=("t","f"), coords = {"t": dt, "f": freq})
        out.to_netcdf('whi'+str(i)+'.nc')

C:\Users\jess\AppData\Local\Temp\ipykernel_57396\363051226.py:13: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  out = xr.DataArray(espd, dims=("t","f"), coords = {"t": dt, "f": freq})
C:\Users\jess\AppData\Local\Temp\ipykernel_57396\363051226.py:13: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the Data

C:\Users\jess\AppData\Local\Temp\ipykernel_57396\363051226.py:13: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  out = xr.DataArray(espd, dims=("t","f"), coords = {"t": dt, "f": freq})


In [66]:
whi7 = xr.open_dataset("whi7.nc")
print(whi7.sel(f=slice(1,10),t=slice("2011-04-03","2011-04-04")))

<xarray.Dataset> Size: 12MB
Dimensions:                        (t: 58481, f: 50)
Coordinates:
  * t                              (t) datetime64[ns] 468kB 2011-04-03T00:00:...
  * f                              (f) float32 200B 1.953 2.116 ... 9.766 9.928
Data variables:
    __xarray_dataarray_variable__  (t, f) float32 12MB ...
